In [2]:
# GENERAL IMPORTS
import cv2
from matplotlib import pyplot as plt
import numpy as np
# from pprint import pprint
# from collections import deque

# FILE IMPORTS
from config_file import Config

In [ ]:
def check_img_channels(img1, img2) -> bool:
    if len(img1.shape) != len(img2.shape):
        raise ValueError("The two images mush have the same numbero of channels: img1 shape = {} and img2 shape = {}".format(img1.shape, img2.shape))
    elif len(img1.shape) == 3:
        if img1.shape[-1] == 3:
            rgb_flag = True
        elif img1.shape[-1] == 1:
            rgb_flag = False
        else:
            raise ValueError("The images can either have 3 channels (RGB) or 1 (grayscale): # of channels of img1 = {}".format(img1.shape[-1]))
        
    return rgb_flag


def SAD(img1, img2, search_area, k, disparity_range) -> float:
    # compute the Sum of Absolute Distances between intensities of the two images
    # in disparity map computation you only need to compare the left pixel (and patch around it) with an horizontal segment in the right image (the disparity_range)
    i_min, i_max, j_min, j_max = search_area    

    disparity_map = np.zeros((i_max-i_min, j_max-j_min, len(disparity_range)))

    rgb_flag = check_img_channels(img1, img2)

    h, w = img1.shape[:2]
    for i in h:
        for j in w:

            for d in disparity_range:
                
                if rgb_flag:
                    window1 = img1[i-k:i+k+1, j-k:j+k+1,            :]
                    window2 = img1[i-k:i+k+1, (j+d)-k:(j+d)+k+1,    :]

                else:
                    window1 = img1[i-k:i+k+1, j-k:j+k+1]
                    window2 = img1[i-k:i+k+1, (j+d)-k:(j+d)+k+1]

                disparity_map[i, j, d] = np.sum(np.abs(window1 - window2))
    
    return np.max(disparity_map, axis=-1)

